In [8]:
import tabula
import pandas as pd
#sequencia de tabelas = 10-11,12-13,14-28,29-38,39-55,56-71,72-85
tabela1 = tabula.read_pdf(
    r"C:\Users\Orcadius\OneDrive\Desktop\work\Hackaton Solutions BI\data\raw\eOuve - Limeira.pdf",
      pages="10-11",
      #stream=True, #gerou inumeras ocorrencias de NaN (not a number)
        lattice=True,#demonstrou melhor resultado em ler por conta de ser feito para tabelas com grade visivel
        multiple_tables=False,
      encoding = "latin1"
      
      
      );

tabela2 = tabula.read_pdf(
    r"C:\Users\Orcadius\OneDrive\Desktop\work\Hackaton Solutions BI\data\raw\eOuve - Limeira.pdf",
      pages="12-13",
        lattice=True,
        multiple_tables=False,
      encoding = "latin1"
      
      
      );


tabela3 = tabula.read_pdf(
    r"C:\Users\Orcadius\OneDrive\Desktop\work\Hackaton Solutions BI\data\raw\eOuve - Limeira.pdf",
      pages="14-28",
        lattice=True,
        multiple_tables=False,
      encoding = "latin1"
      
      
      );


tabela4 = tabula.read_pdf(
    r"C:\Users\Orcadius\OneDrive\Desktop\work\Hackaton Solutions BI\data\raw\eOuve - Limeira.pdf",
      pages="29-38",
        lattice=True,
        multiple_tables=False,
      encoding = "latin1"
      
      
      );

tabela5 = tabula.read_pdf(
    r"C:\Users\Orcadius\OneDrive\Desktop\work\Hackaton Solutions BI\data\raw\eOuve - Limeira.pdf",
      pages="39-55",
        lattice=True,
        multiple_tables=False,
      encoding = "latin1"
      
      
      );

tabela6 = tabula.read_pdf(
    r"C:\Users\Orcadius\OneDrive\Desktop\work\Hackaton Solutions BI\data\raw\eOuve - Limeira.pdf",
      pages="56-71",
        lattice=True,
        multiple_tables=False,
      encoding = "latin1"
      
      
      );


tabela7 = tabula.read_pdf(
    r"C:\Users\Orcadius\OneDrive\Desktop\work\Hackaton Solutions BI\data\raw\eOuve - Limeira.pdf",
      pages="72-85",
        lattice=True,
        multiple_tables=False,
      encoding = "latin1"
      
      
      );



colunasAssuntosSituacao = ['Assunto',	'Aberto',	'Agendado',	'Cancelado',	'Concluído'	,'Em andamento',	'Pendente',	'Total']
colunasSecretariaSituacao = ['Secretaria',	'Aberto',	'Agendado',	'Cancelado',	'Concluído', 'Em andamento',	'Pendente',	'Total']
colunasSecretariaTipo = ['Secretaria','Denúncia',	'Doação',	'Elogio',	'Informação',	'Reclamação',	'Simplifique'	,'Solicitação',	'Sugestão',	'Total']
colunasAssuntosTipo = ['Assunto','Denúncia',	'Doação',	'Elogio',	'Informação',	'Reclamação',	'Simplifique'	,'Solicitação',	'Sugestão',	'Total']
colunasBairroTipo = ['Bairro','Denúncia',	'Doação',	'Elogio',	'Informação',	'Reclamação',	'Simplifique'	,'Solicitação',	'Sugestão',	'Total']
colunasBairroSituacao = ['Bairro',	'Aberto',	'Agendado',	'Cancelado',	'Concluído', 'Em andamento',	'Pendente',	'Total']

modeloSituacao = ['Aberto',	'Agendado',	'Cancelado',	'Concluído'	,'Em andamento',	'Pendente',	'Total']
modeloTipo = ['Denúncia',	'Doação',	'Elogio',	'Informação',	'Reclamação',	'Simplifique'	,'Solicitação',	'Sugestão',	'Total']

valoresSituacao = {'Aberto' : int,	'Agendado' : int,	'Cancelado': int,	'Concluído': int	,'Em andamento': int,	'Pendente': int,'Total' : int}
valoresTipo = {'Denúncia': int,	'Doação': int,	'Elogio': int,	'Informação': int,	'Reclamação': int,	'Simplifique': int	,'Solicitação': int,	'Sugestão': int,'Total' : int}






def limparTexto(tabela,coluna):
    tabela[coluna] = tabela[coluna].str.replace('\r',' ',regex = False)

    return tabela
    


def limparValores(tabela,listaDeColunas):
    for coluna in listaDeColunas:
        tabela[coluna] = tabela[coluna].astype(str).str.replace('.','',regex = False).str.replace(',','.',regex = False)
        tabela[coluna] = pd.to_numeric(tabela[coluna], errors='coerce')
        tabela[coluna] = tabela[coluna].fillna(0).astype('Int64')

    return tabela

#para evitar a criacao de outra lista de colunas sem o parametro Total, o metodo abaixo tem a variavel colunasParaSomar que recebe todas as colunas -1 que é a Total
def calcularTotal(tabela,modeloColunas):
    colunasParaSomar = modeloColunas[:-1]
    tabela['Total'] = tabela[colunasParaSomar].sum(axis=1)
    return tabela




tabela1[0].columns = colunasSecretariaTipo
tabela2[0].columns = colunasSecretariaSituacao
tabela3[0].columns = colunasAssuntosTipo
tabela4[0].columns = colunasAssuntosSituacao
tabela5[0].columns = colunasBairroTipo
tabela6[0].columns = colunasBairroSituacao
tabela7[0].columns = colunasSecretariaTipo


#apagar ultima linha
tabela1[0].drop(tabela1[0].tail(1).index,inplace=True)
tabela2[0].drop(tabela2[0].tail(1).index,inplace=True)
tabela3[0].drop(tabela3[0].tail(1).index,inplace=True)
tabela4[0].drop(tabela4[0].tail(1).index,inplace=True)
tabela5[0].drop(tabela5[0].tail(1).index,inplace=True)
tabela6[0].drop(tabela6[0].tail(1).index,inplace=True)
tabela7[0].drop(tabela7[0].tail(1).index,inplace=True)


tabela1[0] = limparValores(tabela1[0],modeloTipo)
tabela2[0] = limparValores(tabela2[0],modeloSituacao)
tabela3[0] = limparValores(tabela3[0],modeloTipo)
tabela4[0] = limparValores(tabela4[0],modeloSituacao)
tabela5[0] = limparValores(tabela5[0],modeloTipo)
tabela6[0] = limparValores(tabela6[0],modeloSituacao)
tabela7[0] = limparValores(tabela7[0],modeloTipo)


tabela1[0] = calcularTotal(tabela1[0],modeloTipo)
tabela2[0] = calcularTotal(tabela2[0],modeloSituacao)
tabela3[0] = calcularTotal(tabela3[0],modeloTipo)
tabela4[0] = calcularTotal(tabela4[0],modeloSituacao)
tabela5[0] = calcularTotal(tabela5[0],modeloTipo)
tabela6[0] = calcularTotal(tabela6[0],modeloSituacao)
tabela7[0] = calcularTotal(tabela7[0],modeloTipo)



tabela1[0] = limparTexto(tabela1[0],'Secretaria')
tabela2[0] = limparTexto(tabela2[0],'Secretaria')
tabela3[0] = limparTexto(tabela3[0],'Assunto')
tabela4[0] = limparTexto(tabela4[0],'Assunto')
tabela5[0] = limparTexto(tabela5[0],'Bairro')
tabela6[0] = limparTexto(tabela6[0],'Bairro')
tabela7[0] = limparTexto(tabela7[0],'Secretaria')

#o codigo abaixo indica que praticamente todo o conteudo foi lido como String(object)
#display(tabela1[0].dtypes)
#display(tabela2[0].dtypes) 
#display(tabela3[0].dtypes) 
#display(tabela4[0].dtypes) 
#display(tabela5[0].dtypes) 
#display(tabela6[0].dtypes) 
#display(tabela7[0].dtypes) 


listaDeTabelas = [tabela1[0],tabela2[0],tabela3[0],tabela4[0],tabela5[0],tabela6[0],tabela7[0]]

def tabelasParaCsv(listacomTabelas):
    n = 1
    for tabela in listacomTabelas:
        tabela.to_csv(f'Tabela{n}_TratadaHackaton.csv',mode = 'w',index = False, header = True)
        n = n + 1

tabelasParaCsv(listaDeTabelas)

display(tabela1[0])
display(tabela2[0])
display(tabela3[0])
display(tabela4[0])
display(tabela5[0])
display(tabela6[0])
display(tabela7[0])




,Secretaria,Denúncia,Doação,Elogio,Informação,Reclamação,Simplifique,Solicitação,Sugestão,Total
0,Animais,0,0,0,0,0,0,0,0,0
1,Denúncia - Descarte Irregular,172,0,0,0,0,0,150,0,322
2,Direitos Humanos - Denúncias de discriminação ...,1,0,0,0,1,0,0,0,2
3,LGPD - Lei Geral de Proteção de Dados,17,0,0,0,0,0,30,0,47
4,PAT - Posto de Atendimento do Trabalhador,0,0,0,0,0,0,0,0,0
5,Secretaria de Agronegócios e Áreas Rurais,2,0,0,0,1,0,326,2,331
6,Secretaria de Comunicação,44,0,15,10267,115,0,24,37,10502
7,Secretaria de Esportes e Lazer,0,0,0,0,1,0,3,95,99
8,Secretaria de Fazenda,43,0,0,0,10,0,494,3,550
9,Secretaria de Meio Ambiente e Saneamento,139,0,0,0,4,0,468,3,614


,Secretaria,Aberto,Agendado,Cancelado,Concluído,Em andamento,Pendente,Total
0,Animais,0,0,0,0,0,0,0
1,Denúncia - Descarte Irregular,61,0,0,227,33,1,322
2,Direitos Humanos - Denúncias de discriminação ...,0,0,0,2,0,0,2
3,LGPD - Lei Geral de Proteção de Dados,0,0,40,7,0,0,47
4,PAT - Posto de Atendimento do Trabalhador,0,0,0,0,0,0,0
5,Secretaria de Agronegócios e Áreas Rurais,2,0,65,179,84,1,331
6,Secretaria de Comunicação,1,0,15,10482,4,0,10502
7,Secretaria de Esportes e Lazer,4,0,1,93,1,0,99
8,Secretaria de Fazenda,37,0,4,509,0,0,550
9,Secretaria de Meio Ambiente e Saneamento,121,0,2,452,37,2,614


,Assunto,Denúncia,Doação,Elogio,Informação,Reclamação,Simplifique,Solicitação,Sugestão,Total
0,Animais - Outros,0,0,0,0,0,0,0,0,0
1,Denúncia - Descarte Irregular - DESCARTE IRREG...,172,0,0,0,0,0,150,0,322
2,Denúncia - Descarte Irregular - Outros,0,0,0,0,0,0,0,0,0
3,Direitos Humanos - Denúncias de discriminação ...,0,0,0,0,0,0,0,0,0
4,Direitos Humanos - Denúncias de discriminação ...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
132,Secretaria dos Transportes e Mobilidade Urbana...,9,0,2,0,9,0,12,449,481
133,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,59,1,60
134,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,20,0,20
135,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,26,1,27


,Assunto,Aberto,Agendado,Cancelado,Concluído,Em andamento,Pendente,Total
0,Animais - Outros,0,0,0,0,0,0,0
1,Denúncia - Descarte Irregular - DESCARTE IRREG...,61,0,0,227,33,1,322
2,Denúncia - Descarte Irregular - Outros,0,0,0,0,0,0,0
3,Direitos Humanos - Denúncias de discriminação ...,0,0,0,0,0,0,0
4,Direitos Humanos - Denúncias de discriminação ...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
136,Secretaria dos Transportes e Mobilidade Urbana...,1,0,1,422,50,7,481
137,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,54,5,1,60
138,Secretaria dos Transportes e Mobilidade Urbana...,1,0,0,17,2,0,20
139,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,27,0,0,27


,Bairro,Denúncia,Doação,Elogio,Informação,Reclamação,Simplifique,Solicitação,Sugestão,Total
0,Anavec,19,0,0,0,1,0,203,4,227
1,Área Rural de Limeira,29,0,0,1,1,0,301,4,336
2,Bairro da Geada,9,0,0,1,0,0,21,0,31
3,Bairro dos Frades,0,0,0,0,0,0,9,0,9
4,Bairro dos Loiolas,0,0,0,0,2,0,38,0,40
...,...,...,...,...,...,...,...,...,...,...
262,Vila São João,23,0,0,0,1,0,206,6,236
263,Vila São Luiz,4,0,0,0,3,0,148,5,160
264,Vila São Roque,1,0,0,0,0,0,81,0,82
265,Vila Solar,0,0,0,0,0,0,35,0,35


,Bairro,Aberto,Agendado,Cancelado,Concluído,Em andamento,Pendente,Total
0,Anavec,8,0,3,192,23,1,227
1,Área Rural de Limeira,19,0,44,203,69,1,336
2,Bairro da Geada,1,0,0,26,4,0,31
3,Bairro dos Frades,0,0,2,6,1,0,9
4,Bairro dos Loiolas,13,0,3,18,6,0,40
...,...,...,...,...,...,...,...,...
264,Vila São João,42,0,2,162,30,0,236
265,Vila São Luiz,14,0,2,130,14,0,160
266,Vila São Roque,5,0,1,75,1,0,82
267,Vila Solar,0,0,1,30,4,0,35


,Secretaria,Denúncia,Doação,Elogio,Informação,Reclamação,Simplifique,Solicitação,Sugestão,Total
0,Animais - Outros,0,0,0,0,0,0,0,0,0
1,Denúncia - Descarte Irregular - DESCARTE IRREG...,27,0,0,0,0,0,23,0,50
2,Denúncia - Descarte Irregular - Outros,0,0,0,0,0,0,0,0,0
3,Direitos Humanos - Denúncias de discriminação ...,0,0,0,0,0,0,0,0,0
4,Direitos Humanos - Denúncias de discriminação ...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
135,Secretaria dos Transportes e Mobilidade Urbana...,2,0,0,0,3,0,1,24,30
136,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,1,0,1
137,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,5,0,5
138,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,2,0,2
